# Transfer Learning with TensorFlow Part 3: Scaling up (Food Vision Mini)

Using 101 classes. 

Goal: Beat original food 101 paper accuracy : 50.76%

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
## Creating helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-01-13 15:36:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-13 15:36:52 (76.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import helper functions
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,compare_historys,walk_through_dir

## 101 Food Classes: working with less data



In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data('101_food_classes_10_percent.zip')

train_dir = '101_food_classes_10_percent/train/'
test_dir = '101_food_classes_10_percent/test/'

--2023-01-13 15:36:59--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   165MB/s    in 11s     

2023-01-13 15:37:09 (146 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [5]:
# How many images/classes
walk_through_dir('101_food_classes_10_percent')

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/train'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/spring_rolls'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/oysters'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/huevos_rancheros'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/lasagna'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/bibimbap'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/guacamole'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/spaghetti_bolognese'.
There are 0 directories and 75 images in '101_food_classes_10_percent/train/fish_and_chips'.
There are 0 directories and 75 images in '

In [6]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224,224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode='categorical',
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                               label_mode='categorical',
                                                                               image_size=IMG_SIZE,
                                                                               shuffle=False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big dog model with trainsfer learning on 10% of 101 food classes

Steps we are going to take

1. Create a ModelCheckpoint callback
2. Create a data augmentation layer
3. Build a headless(no top layers) Functional EfficientNetB0 backboned-model (Will create our own output layer)
4. Compile 
5. Feature extract for 5 epochs (validate on 15% of testing data for each epoch)


In [7]:
# Create checkpoint callback
checkpoint_path = '101_classes_10_percent_data_model_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

In [11]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
    # preprocessing.Rescaling(1/255.) 
],name='data_augmentation')


In [13]:
# Setup the bease model and freeze its layers (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with tranable top layers
inputs = layers.Input(shape=(224,224,3),name='input_layer')
x = data_augmentation(inputs)
x = base_model(x,training=False) 
x = layers.GlobalAveragePooling2D(name='global_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names),activation='softmax',name='output_layer')(x)
model = tf.keras.Model(inputs,outputs)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224)         0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_avg_pool_layer (Glob  (None, 1280)             0         
 alAveragePooling2D)                                             
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainab

In [15]:
# Compile the model
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [ ]:
from tensorflow.python import test
# Fit the model
history_0 = model.fit(train_data_all_10_percent,
                      epochs = 5,
                      validation_data = test_data,
                      validation_steps = int(0.15*len(test_data)),
                      callbacks = [checkpoint_callback])

Epoch 1/5


237/237 [==============================] - ETA: 0s - loss: 3.3684 - accuracy: 0.2702

237/237 [==============================] - 604s 3s/step - loss: 3.3684 - accuracy: 0.2702 - val_loss: 2.4113 - val_accuracy: 0.4831
Epoch 2/5
237/237 [==============================] - ETA: 0s - loss: 2.2059 - accuracy: 0.4964

237/237 [==============================] - 595s 3s/step - loss: 2.2059 - accuracy: 0.4964 - val_loss: 1.9557 - val_accuracy: 0.5368
Epoch 3/5
237/237 [==============================] - ETA: 0s - loss: 1.8230 - accuracy: 0.5711

237/237 [==============================] - 580s 2s/step - loss: 1.8230 - accuracy: 0.5711 - val_loss: 1.7928 - val_accuracy: 0.5540
Epoch 4/5
180/237 [=====================>........] - ETA: 1:37 - loss: 1.6076 - accuracy: 0.6113

In [ ]:
# Evaluate the model
feature_extraction_results = model.evaluate(test_data)
feature_extraction_results

In [ ]:
plot_loss_curves(history_0)

## Fine-tuning

In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable = True

# Refreeze every layer except the last 5
for layer in base_model.layers[:-5]:
  layers.trainable = False

In [ ]:
# Recompile model with lowe learning 
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(lr=0.0001), # reduce by 10x
    metrics = ['accuracy']
)

In [ ]:
# What layers in the model are trainable?
for layer in model.layer:
  print(layer.name,layer.trainable)

In [ ]:
# Check which layers are trainable in our base model
for layer_number,layer in enumerate(model.layers[2].layers):
  print(layer_number,layer.name,layer.trainable)

In [ ]:
# Fine-tune for 5 more epochs
fine_tune_epochs = 10 # model has already done 5 epochs (feature extraction)

# Fine-tune our model
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15*len(test_data)),
                                                     initial_epoch=history_0.epoch[-1])

In [ ]:
# Evaluate on the whole test data
all_classes_10_percent_fine_tune_results = model.evaluate(test_data)
all_classes_10_percent_fine_tune_results

In [ ]:
# Compare the histories of feature extraction model with fine-tuning model
compare_historys(original_history=history_0,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)

## Saving and loading our model

To use our model in an external application, we will need to save it and export it somewhere

In [ ]:
# Save our fine-tuned model
model.save('drive/MyDrive/Tensorflow_Certificate/101_food_classes_10_percent_saved_big_dog_model')

In [ ]:
# Load saved model
loaded_model = tf.keras.model.load_model('drive/MyDrive/Tensorflow_Certificate/101_food_classes_10_percent_saved_big_dog_model')

In [ ]:
# Evaluate loaded model and compare to previous model
loaded_model_results = loaded_model.evaluate(test_data)
loaded_model_results

In [ ]:
# The result above should be similar to results below
all_classes_10_percent_fine_tune_results

## Evlauating the performance of the big dog model across all different classes

Let's make some predictioins, visualize them and then later find out which predictions were the 'most' wrong.

In [ ]:
import tensorflow as tensorflow
# Donwload pretrained model pretrained earlier
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [ ]:
# Unzip the data
unzip_data('06_101_food_class_10_percent_saved_big_dog_model.zip')

In [ ]:
# Loading the model
model = tf.keras.models.load_model('06_101_food_class_10_percent_saved_big_dog_model')